In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00


In [ ]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load pandas dataset by nodeid and stored in the `data` and `type` variable

In [ ]:
import requests
import os
import pandas as pd

data = []
types = []

# Get Api URL from user
URL = input("Website URL (Default http://macwater.cas.mcmaster.ca): ")
if (URL == ''):
    URL = 'http://macwater.cas.mcmaster.ca'
if (URL[-1] != '/'):
    URL += '/'

# Get location information from server
infoResponse = requests.get(url = (URL + 'sensor/info'))
infoData = infoResponse.json()["locations"]

# Show all node infomations
display(pd.DataFrame(infoData, columns=["NodeID", "Name", "Location", "Time"]))

# Ask user to enter a NodeID
nodeid = input("\nSelect and enter a NodeID: ")

# Loop through each location
for location in infoData:
    if (location[0] != nodeid):
      continue
    print("\nNodeID: "+location[0]+", Name: "+location[1])

    # Get the type information for this node
    types = (requests.get(url = (URL + 'sensor/type'), params = {'id': location[0]})).json()["types"]
    print("\nTypes: ")
    print(types)

    # Get the location history for this node
    allLocations = (requests.get(url = (URL + 'sensor/info'), params = {'id': location[0]})).json()["locations"]
    print("\nAllLocations: ")
    print(allLocations)

    # If there are sensors
    if (len(types) != 0):
        # Get the sensor data for every sensor
        for i in types:
            # Get sensor data from api
            currData = (requests.get(url = (URL + 'sensor/data'), params = {'id': location[0], 'type': i})).json()["data"]
            # Addend current data to `data` variable
            data.append(pd.DataFrame(currData, columns=["Time", i]))
            # Convert unixtimestamp(ns) to datetime
            data[len(data)-1]['Time'] = pd.to_datetime(data[len(data)-1]['Time'].astype(float)/1e9,unit='s')
            # Convert sensor data to integer
            data[len(data)-1][i] = pd.to_numeric(data[len(data)-1][i])
print("\nDone")

Website URL (Default http://macwater.cas.mcmaster.ca): 


,NodeID,Name,Location,Time
0,102,M5,"43.080139,-80.077011",0
1,6,M4,"43.052223,-80.128875",0



Select and enter a NodeID: 102

NodeID: 102, Name: M5

Types: 
['Air_Temperature', 'Conductivity', 'Dissolved_Oxygen', 'Humidity', 'Turbidity', 'Water_Temperature', 'pH']

AllLocations: 
[['0', '43.080139,-80.077011']]

Done


Display graph

In [ ]:
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go

print("NodeID: " + nodeid + "\n")
# Copy data from global variables
c_data = data
c_types = types

sub_tab=[widgets.Output() for i in range(len(c_data))]
tab = widgets.Tab(sub_tab)
for i in range (len(c_data)):
    tab.set_title(i,c_types[i]) # Tab title
    with sub_tab[i]:
      df = c_data[i]

      # Plot data
      fig = px.line(df, x='Time', y=c_types[i], title=c_types[i])
      fig.update_xaxes(
          rangeslider_visible=True,
          rangeselector=dict(
              buttons=list([
                  dict(count=1, label="1d", step="day", stepmode="backward"),
                  dict(count=5, label="5d", step="day", stepmode="backward"),
                  dict(count=15, label="15d", step="day", stepmode="backward"),
                  dict(count=1, label="1m", step="month", stepmode="backward"),
                  dict(count=6, label="6m", step="month", stepmode="backward"),
                  dict(count=1, label="YTD", step="year", stepmode="todate"),
                  dict(count=1, label="1y", step="year", stepmode="backward"),
                  dict(step="all")
              ])
          )
      )
      fig.show()
display(tab)

NodeID: 102



Data table

In [ ]:
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go

print("NodeID: " + nodeid + "\n")
# Copy data from global variables
c_data = data
c_types = types

sub_tab=[widgets.Output() for i in range(len(c_data))]
tab = widgets.Tab(sub_tab)
for i in range (len(c_data)):
    tab.set_title(i,c_types[i]) # Tab title
    with sub_tab[i]:
        df = c_data[i]

        # Show dataset
        fig = go.Figure(data=[go.Table(
            header=dict(values=list(df.columns),
                        align='left'),
            cells=dict(values=[df.Time, df[c_types[i]]],
                      align='left'))
        ])
        fig.show()
        #display(df)
display(tab)

NodeID: 102



Histogram

In [ ]:
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
import numpy as np # linear algebra
import matplotlib.pyplot as plt # plotting

print("NodeID: " + nodeid + "\n")
# Copy data from global variables
c_data = data
c_types = types

sub_tab=[widgets.Output() for i in range(len(c_data))]
tab = widgets.Tab(sub_tab)
for i in range (len(c_data)):
    tab.set_title(i,c_types[i]) # Tab title
    with sub_tab[i]:
        df = c_data[i]

        # Plot data
        plt.style.use("ggplot")
        plt.figure()
        depth = df[(df[types[i]] < 10000) & (df[c_types[i]] >= 0)][c_types[i]]
        depth.plot(kind="hist", stacked=True, bins=50)
        plt.title(c_types[i]+" Histogram")
        plt.xlabel(c_types[i])
        plt.show()
display(tab)

NodeID: 102

